In [2]:
import torch
from torchvision import (
    datasets,
    transforms
)
from torch.utils import data

import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use("seaborn-v0_8")
plt.rcParams["font.family"] = "monospace"

In [3]:
if torch.cuda.is_available():
    print("CUDA found, bounding...")
    DEVICE = torch.device('cuda:0')
else:
    print("NO CUDA found, bounding to CPU...")
    DEVICE = torch.device('cpu')

CUDA found, bounding...


In [4]:
torch.cuda.get_device_properties(DEVICE)

_CudaDeviceProperties(name='NVIDIA GeForce RTX 4060 Laptop GPU', major=8, minor=9, total_memory=8187MB, multi_processor_count=24)

In [5]:
DTYPE = torch.float32